In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import os
import cv2
import numpy
import imageio
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from keras import backend as K
import sys
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [0]:
image_rows, image_columns, image_depth = 64, 64, 18

In [0]:
training_list = []
painpath = '/content/drive/My Drive/Final Training Data/pain'
Nopainpath = '/content/drive/My Drive/Final Training Data/Nopain'

In [0]:
directorylisting = os.listdir(painpath)

In [0]:
for video in os.listdir(painpath):
  videopath = os.path.join(painpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  training_list.append(videoarray)

In [0]:
len(training_list)

25

In [0]:
directorylisting = os.listdir(Nopainpath)

In [0]:
for video in os.listdir(Nopainpath):
  videopath = os.path.join(Nopainpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  training_list.append(videoarray)

In [0]:
len(training_list)

50

In [0]:
training_list = numpy.asarray(training_list)
trainingsamples = len(training_list)
traininglabels = numpy.zeros((trainingsamples, ), dtype = int)

In [0]:
traininglabels[0:25] = 0
traininglabels[25:50] = 1

In [0]:
traininglabels = np_utils.to_categorical(traininglabels, 2)

In [0]:
training_data = [training_list, traininglabels]
(trainingframes, traininglabels) = (training_data[0], training_data[1])
training_set = numpy.zeros((trainingsamples, 1, image_rows, image_columns, image_depth))

In [0]:
training_set.shape

(50, 1, 64, 64, 18)

In [0]:
for h in range(trainingsamples):
    training_set[h][0][:][:][:] = trainingframes[h, :, :, :]

In [0]:
training_set = training_set.astype('float32')
training_set -= numpy.mean(training_set)
training_set /= numpy.max(training_set)

In [0]:
model = Sequential()
model.add(Convolution3D(4, (3, 3, 15), input_shape=(1, image_rows, image_columns, image_depth), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, init='normal'))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 4, 62, 62, 4)      544       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 20, 20, 1)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 20, 20, 1)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 3202      
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 3,746
Trainable params: 3,746
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="normal")`
  


In [0]:
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_set, traininglabels, test_size=0.3, random_state=4)

In [0]:
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), nb_epoch = 15, shuffle=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0402 19:03:46.580770 140513863133056 deprecation.py:323] From /tensorflow-1.15.2/python2.7/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0402 19:03:46.679775 140513863133056 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0402 19:03:46.697853 140513863133056 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 35 samples, validate on 15 samples
Epoch 1/15
35/35 [==============================] - 3s 77ms/step - loss: 0.7682 - acc: 0.3429 - val_loss: 0.6919 - val_acc: 0.5333
Epoch 2/15
35/35 [==============================] - 2s 62ms/step - loss: 0.8058 - acc: 0.4286 - val_loss: 0.5363 - val_acc: 0.8000
Epoch 3/15
35/35 [==============================] - 2s 61ms/step - loss: 0.6651 - acc: 0.4571 - val_loss: 0.4824 - val_acc: 1.0000
Epoch 4/15
35/35 [==============================] - 2s 63ms/step - loss: 0.5970 - acc: 0.6857 - val_loss: 0.4507 - val_acc: 0.9333
Epoch 5/15
35/35 [==============================] - 2s 62ms/step - loss: 0.5620 - acc: 0.6571 - val_loss: 0.4052 - val_acc: 0.9333
Epoch 6/15
35/35 [==============================] - 2s 61ms/step - loss: 0.5259 - acc: 0.7714 - val_loss: 0.3463 - val_acc: 0.9333
Epoch 7/15
35/35 [==============================] - 2s 61ms/step - loss: 0.4722 - acc: 0.8000 - val_loss: 0.3099 - val_acc: 0.9333
Epoch 8/15
35/35 [=====================

In [0]:
model.save('/content/drive/My Drive/Research project/Pre-trained_model_0.3%.h5')

In [0]:
predictions = model.predict(validation_images)
predictions_labels = numpy.argmax(predictions, axis=1)
validation_labels = numpy.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)

[[8 1]
 [0 6]]
